<a href="https://colab.research.google.com/github/ZS4MLDL/hugging_face_code/blob/main/finetune_ner_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import pandas as pd
import json
import requests

In [10]:
train = pd.read_csv("https://raw.githubusercontent.com/laxmimerit/All-CSV-ML-Data-Files-Download/refs/heads/master/mit_restaurant_search_ner/train.bio", sep="\t", header=None)
train.head()

,0,1
0,B-Rating,2
1,I-Rating,start
2,O,restaurants
3,O,with
4,B-Amenity,inside


In [14]:
response = requests.get("https://raw.githubusercontent.com/laxmimerit/All-CSV-ML-Data-Files-Download/refs/heads/master/mit_restaurant_search_ner/train.bio")
response = response.text.splitlines()
response

['B-Rating\t2',
 'I-Rating\tstart',
 'O\trestaurants',
 'O\twith',
 'B-Amenity\tinside',
 'I-Amenity\tdining',
 '',
 'O\t34',
 '',
 'B-Rating\t5',
 'I-Rating\tstar',
 'O\tresturants',
 'B-Location\tin',
 'I-Location\tmy',
 'I-Location\ttown',
 '',
 'O\t98',
 'B-Restaurant_Name\thong',
 'I-Restaurant_Name\tkong',
 'O\trestaurant',
 'B-Price\treasonable',
 'O\tprices',
 '',
 'O\ta',
 'O\tgreat',
 'O\tlunch',
 'O\tspot',
 'O\tbut',
 'B-Hours\topen',
 'I-Hours\ttill',
 'I-Hours\t2',
 'I-Hours\ta',
 'I-Hours\tm',
 'B-Restaurant_Name\tpassims',
 'I-Restaurant_Name\tkitchen',
 '',
 'O\ta',
 'O\tplace',
 'O\tthat',
 'O\tserves',
 'B-Dish\tsoft',
 'I-Dish\tserve',
 'I-Dish\tice',
 'I-Dish\tcream',
 '',
 'O\ta',
 'O\trestaurant',
 'O\tthat',
 'O\tis',
 'B-Rating\tgood',
 'B-Amenity\tfor',
 'I-Amenity\tgroups',
 '',
 'O\ta',
 'B-Dish\tsalad',
 'O\twould',
 'O\tmake',
 'O\tmy',
 'O\tday',
 '',
 'O\ta',
 'B-Cuisine\tsmoothie',
 'O\twould',
 'O\thit',
 'O\tthe',
 'O\tspot',
 '',
 'O\ta',
 'B-Dish\ts

In [16]:
temp_tokens = []
temp_tags = []
train_tokens = []
train_tags = []
for line in response:
  if line != "":
    tag, token = line.strip().split('\t')
    temp_tags.append(tag)
    temp_tokens.append(token)
  else:
    train_tokens.append(temp_tokens)
    train_tags.append(temp_tags)

    temp_tags, temp_tokens = [], []


len(train_tokens), len(train_tags)

(7659, 7659)

In [20]:
from datasets import Dataset, DatasetDict

df = pd.DataFrame({'tokens': train_tokens, 'ner_tags_str': train_tags})
train = Dataset.from_pandas(df)
train

Dataset({
    features: ['tokens', 'ner_tags_str'],
    num_rows: 7659
})

In [21]:
#dataset = DatasetDict({'train':train})

In [22]:
response = requests.get("https://raw.githubusercontent.com/laxmimerit/All-CSV-ML-Data-Files-Download/refs/heads/master/mit_restaurant_search_ner/test.bio")
response = response.text.splitlines()
response

['O\ta',
 'B-Rating\tfour',
 'I-Rating\tstar',
 'O\trestaurant',
 'B-Location\twith',
 'I-Location\ta',
 'B-Amenity\tbar',
 '',
 'O\tany',
 'B-Cuisine\tasian',
 'O\tcuisine',
 'B-Location\taround',
 '',
 'O\tany',
 'B-Cuisine\tbbq',
 'O\tplaces',
 'B-Hours\topen',
 'I-Hours\tbefore',
 'I-Hours\t5',
 'B-Location\tnearby',
 '',
 'O\tany',
 'B-Location\tdancing',
 'I-Location\testablishments',
 'O\twith',
 'B-Price\treasonable',
 'O\tpricing',
 '',
 'O\tany',
 'O\tgood',
 'B-Price\tcheap',
 'B-Cuisine\tgerman',
 'O\trestaurants',
 'B-Location\tnearby',
 '',
 'O\tany',
 'B-Rating\tgood',
 'B-Cuisine\tice',
 'I-Cuisine\tcream',
 'I-Cuisine\tparlors',
 'B-Location\taround',
 '',
 'O\tany',
 'B-Rating\tgood',
 'O\tplace',
 'O\tto',
 'O\tget',
 'O\ta',
 'B-Dish\tpie',
 'O\tat',
 'O\tan',
 'B-Price\taffordable',
 'O\tprice',
 '',
 'O\tany',
 'O\tgood',
 'B-Cuisine\tvegan',
 'O\tspots',
 'B-Location\tnearby',
 '',
 'O\tany',
 'B-Cuisine\tmexican',
 'O\tplaces',
 'O\thave',
 'O\ta',
 'B-Dish\ttam

In [23]:
temp_tokens = []
temp_tags = []
test_tokens = []
test_tags = []
for line in response:
  if line != "":
    tag, token = line.strip().split('\t')
    temp_tags.append(tag)
    temp_tokens.append(token)
  else:
    test_tokens.append(temp_tokens)
    test_tags.append(temp_tags)

    temp_tags, temp_tokens = [], []


len(test_tokens), len(test_tags)

(1520, 1520)

In [25]:
from datasets import Dataset, DatasetDict

df = pd.DataFrame({'tokens': test_tokens, 'ner_tags_str': test_tags})
test = Dataset.from_pandas(df)
test

Dataset({
    features: ['tokens', 'ner_tags_str'],
    num_rows: 1520
})

In [28]:
dataset = DatasetDict({'train':train,'test':test, 'validation':test})

In [29]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags_str'],
        num_rows: 7659
    })
    test: Dataset({
        features: ['tokens', 'ner_tags_str'],
        num_rows: 1520
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags_str'],
        num_rows: 1520
    })
})

In [34]:
unique_tags = set()
for tag in dataset['train']['ner_tags_str']:
  unique_tags.update(tag)

unique_tags = list(set([x[2:] for x in list(unique_tags) if x!='O']))
unique_tags

['Rating',
 'Cuisine',
 'Amenity',
 'Dish',
 'Price',
 'Restaurant_Name',
 'Hours',
 'Location']

In [35]:
tag2index = {"O":0}

for i, tag in enumerate(unique_tags):
  tag2index[f"B-{tag}"] = len(tag2index)
  tag2index[f"I-{tag}"] = len(tag2index)

index2tag = {v:k for k, v in tag2index.items()}

tag2index, index2tag

({'O': 0,
  'B-Rating': 1,
  'I-Rating': 2,
  'B-Cuisine': 3,
  'I-Cuisine': 4,
  'B-Amenity': 5,
  'I-Amenity': 6,
  'B-Dish': 7,
  'I-Dish': 8,
  'B-Price': 9,
  'I-Price': 10,
  'B-Restaurant_Name': 11,
  'I-Restaurant_Name': 12,
  'B-Hours': 13,
  'I-Hours': 14,
  'B-Location': 15,
  'I-Location': 16},
 {0: 'O',
  1: 'B-Rating',
  2: 'I-Rating',
  3: 'B-Cuisine',
  4: 'I-Cuisine',
  5: 'B-Amenity',
  6: 'I-Amenity',
  7: 'B-Dish',
  8: 'I-Dish',
  9: 'B-Price',
  10: 'I-Price',
  11: 'B-Restaurant_Name',
  12: 'I-Restaurant_Name',
  13: 'B-Hours',
  14: 'I-Hours',
  15: 'B-Location',
  16: 'I-Location'})

In [38]:
dataset = dataset.map(lambda example: {"ner_tags":[tag2index[tag] for tag in example['ner_tags_str']]})

Map:   0%|          | 0/7659 [00:00<?, ? examples/s]

Map:   0%|          | 0/1520 [00:00<?, ? examples/s]

Map:   0%|          | 0/1520 [00:00<?, ? examples/s]

In [39]:
from transformers import AutoTokenizer
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [46]:
input = dataset['train'][2]['tokens']
output = tokenizer(input, is_split_into_words=True)
input , output, output.word_ids()

(['5', 'star', 'resturants', 'in', 'my', 'town'],
 {'input_ids': [101, 1019, 2732, 2717, 4648, 7666, 1999, 2026, 2237, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 [None, 0, 1, 2, 2, 2, 3, 4, 5, None])

In [47]:
tokenizer.convert_ids_to_tokens(output.input_ids)

['[CLS]', '5', 'star', 'rest', '##ura', '##nts', 'in', 'my', 'town', '[SEP]']

In [49]:
def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)

  labels = []
  for i, label in enumerate(examples['ner_tags']):
    word_ids = tokenized_inputs.word_ids(batch_index=i)

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:
      if word_idx is None:
        label_ids.append(-100)
      elif word_idx != previous_word_idx:
        label_ids.append(label[word_idx])
      else:
        label_ids.append(-100)

        prev_word_idx = word_idx

    labels.append(label_ids)

  tokenized_inputs['labels'] = labels

  return tokenized_inputs

In [53]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)
tokenized_dataset

Map:   0%|          | 0/7659 [00:00<?, ? examples/s]

Map:   0%|          | 0/1520 [00:00<?, ? examples/s]

Map:   0%|          | 0/1520 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags_str', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 7659
    })
    test: Dataset({
        features: ['tokens', 'ner_tags_str', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1520
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags_str', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1520
    })
})

In [54]:
tokenized_dataset['train'][2]

{'tokens': ['5', 'star', 'resturants', 'in', 'my', 'town'],
 'ner_tags_str': ['B-Rating',
  'I-Rating',
  'O',
  'B-Location',
  'I-Location',
  'I-Location'],
 'ner_tags': [1, 2, 0, 15, 16, 16],
 'input_ids': [101, 1019, 2732, 2717, 4648, 7666, 1999, 2026, 2237, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 1, 2, 0, 0, 0, 15, 16, 16, -100]}